In [13]:
import pandas as pd
import os
import json
from tqdm import tqdm
import networkx as nx
import semanticscholar as sch
import urllib
import json
import requests
import itertools
from itertools import groupby

In [2]:
cur_path = os.getcwd()
all_paper_data = pd.read_pickle("..\\data\\all_paper_data.pkl")


In [3]:
coauthors = []
for idx,paper in tqdm(all_paper_data.iterrows()):
    coauthors.extend(list(itertools.combinations(paper['AA'],2))) 
coauthors_researchers_df = pd.DataFrame(coauthors, columns=['researcher1','researcher2'])
coauthors_researchers_df

35034it [00:03, 10125.84it/s]


,researcher1,researcher2
0,"{'AuN': 'karen simonyan', 'AuId': 2033942250, ...","{'AuN': 'andrew zisserman', 'AuId': 2469405535..."
1,"{'AuN': 'diederik p kingma', 'AuId': 205560460...","{'AuN': 'jimmy ba', 'AuId': 2097546270, 'AfN':..."
2,"{'AuN': 'dzmitry bahdanau', 'AuId': 2509101809...","{'AuN': 'kyunghyun cho', 'AuId': 2462121670, '..."
3,"{'AuN': 'dzmitry bahdanau', 'AuId': 2509101809...","{'AuN': 'yoshua bengio', 'AuId': 161269817, 'A..."
4,"{'AuN': 'kyunghyun cho', 'AuId': 2462121670, '...","{'AuN': 'yoshua bengio', 'AuId': 161269817, 'A..."
...,...,...
313067,"{'AuN': 'thomas ricatte', 'AuId': 256014509, '...","{'AuN': 'remi gilleron', 'AuId': 1217769045, '..."
313068,"{'AuN': 'thomas ricatte', 'AuId': 256014509, '...","{'AuN': 'marc tommasi', 'AuId': 2086670939, 'A..."
313069,"{'AuN': 'remi gilleron', 'AuId': 1217769045, '...","{'AuN': 'marc tommasi', 'AuId': 2086670939, 'A..."
313070,"{'AuN': 'r anton', 'AuId': 2271936098, 'AfId':...","{'AuN': 'k dmitry', 'AuId': 2516428222, 'AfId'..."


In [ ]:
authors = itertools.combinations(all_paper_data['AA'].explode('event_params').reset_index(drop = True),2)
colleagues = [pair for pair in tqdm(authors) if pair[0]['AuId']!=pair[1]['AuId'] and pair[0]['AfId']==pair[1]['AfId']] 


In [33]:
authors_exploded = list(all_paper_data['AA'].explode('event_params').reset_index(drop = True))
authors = list({v['AuId']:v for v in authors_exploded if v['AfId'] is not None}.values())
authors.sort(key=lambda x:x['AfId'])
colleagues = []
for k,v in groupby(authors,key=lambda x:x['AfId']):
    colleagues.extend(itertools.combinations(list(v),2))
colleagues_researchers_df = pd.DataFrame(colleagues, columns=['researcher1','researcher2'])
colleagues_researchers_df

,researcher1,researcher2
0,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'maria regina justina e estuar', 'AuId..."
1,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'john clifford rosales', 'AuId': 25756..."
2,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'ma regina e estuar', 'AuId': 14622138..."
3,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'luke prudente', 'AuId': 2610175157, '..."
4,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'andrei d coronel', 'AuId': 2140270248..."
...,...,...
4484062,"{'AuN': 'zaiyue yang', 'AuId': 2888925995, 'Af...","{'AuN': 'ke tang', 'AuId': 2148973427, 'AfN': ..."
4484063,"{'AuN': 'zaiyue yang', 'AuId': 2888925995, 'Af...","{'AuN': 'peng yang', 'AuId': 2444236273, 'AfN'..."
4484064,"{'AuN': 'yunwen lei', 'AuId': 2550290020, 'AfN...","{'AuN': 'ke tang', 'AuId': 2148973427, 'AfN': ..."
4484065,"{'AuN': 'yunwen lei', 'AuId': 2550290020, 'AfN...","{'AuN': 'peng yang', 'AuId': 2444236273, 'AfN'..."


In [34]:
reference_df = pd.concat([colleagues_researchers_df, coauthors_researchers_df])
reference_df

,researcher1,researcher2
0,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'maria regina justina e estuar', 'AuId..."
1,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'john clifford rosales', 'AuId': 25756..."
2,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'ma regina e estuar', 'AuId': 14622138..."
3,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'luke prudente', 'AuId': 2610175157, '..."
4,"{'AuN': 'hadrian paulo m lim', 'AuId': 2750994...","{'AuN': 'andrei d coronel', 'AuId': 2140270248..."
...,...,...
4484062,"{'AuN': 'zaiyue yang', 'AuId': 2888925995, 'Af...","{'AuN': 'ke tang', 'AuId': 2148973427, 'AfN': ..."
4484063,"{'AuN': 'zaiyue yang', 'AuId': 2888925995, 'Af...","{'AuN': 'peng yang', 'AuId': 2444236273, 'AfN'..."
4484064,"{'AuN': 'yunwen lei', 'AuId': 2550290020, 'AfN...","{'AuN': 'ke tang', 'AuId': 2148973427, 'AfN': ..."
4484065,"{'AuN': 'yunwen lei', 'AuId': 2550290020, 'AfN...","{'AuN': 'peng yang', 'AuId': 2444236273, 'AfN'..."
